# Chemical Space, QSAR, and Model Pitfalls

## 90-Minute Laboratory

This notebook explores fundamental concepts in computational drug discovery:

1. **Chemical Space Exploration** — How molecules are represented and visualized
2. **Activity Cliffs** — Pairs of structurally similar molecules with drastically different activities
3. **QSAR Modeling** — Quantitative Structure-Activity Relationship prediction
4. **Dataset Splitting Strategies** — Why the choice of train/test split matters dramatically
5. **Model Explainability** — Understanding what drives predictions

### Dataset
We use the **SARS-CoV-2 Main Protease (Mpro)** inhibitor dataset from the COVID Moonshot project:
- Source: [QSAR Activity Cliff Experiments](https://github.com/MarkusFerdinandDablander/QSAR-activity-cliff-experiments)
- Target: IC50 values for Mpro inhibition
- Molecules: ~1000 compounds with experimental measurements

---

## 📋 Exercises Overview

| Exercise | Task | Deliverable |
|----------|------|-------------|
| 1.1 | Explore data and identify censoring threshold | Plot + threshold value |
| 1.2 | Clean data and compute descriptors | Cleaned DataFrame |
| 2.1 | Compute fingerprints and UMAP | UMAP visualization |
| 3.1 | Find activity cliffs | List of cliff pairs + visualization |
| 4.1 | Implement scaffold split | Split function |
| 4.2 | Implement Butina clustering split | Split function |
| 4.3 | Compare 4 splitting strategies | Comparison table + plots |
| 5.1 | Analyze results and identify hardest split | Summary with interpretation |

In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================
# Core scientific stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# RDKit for cheminformatics
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, Descriptors
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.ML.Cluster import Butina
from rdkit.Chem.Draw import MolsToGridImage

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
from xgboost import XGBRegressor

# Dimensionality reduction
import umap

# Display utilities
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

print("All imports successful!")

## 1. Global Parameters

Key configuration values for reproducibility:
- **FP_RADIUS / FP_BITS**: Morgan fingerprint parameters (radius=2 gives ECFP4-like fingerprints)
- **RANDOM_SEED**: For reproducibility across runs

In [ ]:
# ============================================================================
# GLOBAL PARAMETERS
# ============================================================================

CSV_URL = (
    "https://raw.githubusercontent.com/"
    "MarkusFerdinandDablander/QSAR-activity-cliff-experiments/main/"
    "data/postera_sars_cov_2_mpro/molecule_data_clean.csv"
)

FP_RADIUS = 2                 # Morgan fingerprint radius (2 = ECFP4-like)
FP_BITS = 2048                # Number of bits in fingerprint vector
RANDOM_SEED = 42              # For reproducibility

np.random.seed(RANDOM_SEED)

## 2. Data Loading & Cleaning

### Why pKi instead of IC50?

IC50 values span several orders of magnitude (nM to mM), making them difficult to model directly. We convert to **pKi** (negative log of IC50 in molar):

$$\text{pKi} = -\log_{10}(\text{IC50}_M)$$

This transformation:
- Converts multiplicative relationships to additive ones
- Makes the distribution more normal
- Puts values on a more intuitive scale (higher = more potent)

### Censored Measurements

Many compounds have IC50 reported as ">100 μM" (meaning: too weak to measure accurately). These **right-censored** values are problematic because:
1. We don't know the true value — only that it exceeds a threshold
2. Including them as if they were exact measurements biases the model
3. They cluster at the threshold, creating artificial patterns

In [ ]:
# ============================================================================
# PROVIDED HELPER FUNCTIONS
# ============================================================================

def load_dataset(url: str) -> pd.DataFrame:
    """Load the SARS-CoV-2 Mpro inhibitor dataset from GitHub."""
    df = pd.read_csv(url)
    assert "SMILES" in df.columns, "Missing SMILES column"
    assert "f_avg_IC50 [uM]" in df.columns, "Missing IC50 column"
    
    df = df[["SMILES", "f_avg_IC50 [uM]"]].copy()
    df.rename(columns={"f_avg_IC50 [uM]": "IC50_uM"}, inplace=True)
    return df


def ic50_to_pki(df: pd.DataFrame) -> pd.DataFrame:
    """Convert IC50 (μM) to pKi = -log10(IC50 in M)."""
    df = df.copy()
    df["IC50_M"] = df["IC50_uM"] * 1e-6
    df["pKi"] = -np.log10(df["IC50_M"])
    return df


def fingerprints_to_array(fps: list) -> np.ndarray:
    """Convert RDKit fingerprints to numpy array."""
    return np.array([np.array(fp) for fp in fps])

---

## 📝 Exercise 1.1: Explore Data and Identify Censoring Threshold

**Task**: Load the dataset, convert IC50 to pKi, and plot the distribution. Identify the censoring threshold from the data.

**Expected Deliverable**:
1. A histogram of pKi values showing the distribution
2. Identification of the censoring threshold (the IC50 value above which measurements are "censored")
3. Brief explanation of how you identified it

**Hints**:
- Look for suspicious spikes or clustering in the distribution
- Check the maximum IC50 values — what's the pattern?
- The censoring threshold creates an artificial "pile-up" of values

In [ ]:
# ============================================================================
# EXERCISE 1.1: Load and explore the data
# ============================================================================

# Step 1: Load the dataset
df = load_dataset(CSV_URL)
print(f"Loaded {len(df)} molecules")
print(f"IC50 range: {df['IC50_uM'].min():.3f} - {df['IC50_uM'].max():.3f} μM")

# Step 2: Convert to pKi
df = ic50_to_pki(df)

# TODO: Plot the pKi distribution (histogram)
# Hint: Use plt.hist() or sns.histplot()


# TODO: Examine the IC50 values to identify the censoring threshold
# Hint: Look at df["IC50_uM"].value_counts() or df["IC50_uM"].describe()


# TODO: Set your identified censoring threshold
IC50_CENSOR_THRESHOLD = None  # TODO: Replace with the threshold you identified (in μM)

print(f"\n📊 Identified censoring threshold: {IC50_CENSOR_THRESHOLD} μM")

---

## 📝 Exercise 1.2: Clean Data and Compute Descriptors

**Task**: Remove censored measurements, parse SMILES to RDKit molecules, and add molecular descriptors.

**Expected Deliverable**:
1. A cleaned DataFrame with only non-censored molecules
2. RDKit molecule objects in a "mol" column
3. Basic molecular descriptors (MW, logP, HBD, HBA)
4. A histogram showing the pKi distribution AFTER removing censored values

In [ ]:
# ============================================================================
# EXERCISE 1.2: Clean data and compute descriptors
# ============================================================================

def remove_censored_measurements(df: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    Remove compounds with IC50 above the censoring threshold.
    
    TODO: Implement this function
    - Filter out rows where IC50_uM > threshold
    - Print how many molecules were removed
    - Return the filtered DataFrame
    """
    # TODO: Your code here
    pass


def smiles_to_mol(df: pd.DataFrame) -> pd.DataFrame:
    """
    Parse SMILES strings into RDKit molecule objects.
    
    TODO: Implement this function
    - Use Chem.MolFromSmiles() to parse each SMILES
    - Handle invalid SMILES (they return None)
    - Add a "mol" column to the DataFrame
    - Return only rows with valid molecules
    """
    # TODO: Your code here
    pass


def add_basic_descriptors(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate basic molecular descriptors.
    
    TODO: Implement this function
    - Add columns: MW (molecular weight), logP, HBD (H-bond donors), HBA (H-bond acceptors)
    - Use Descriptors.MolWt, Descriptors.MolLogP, Descriptors.NumHDonors, Descriptors.NumHAcceptors
    """
    # TODO: Your code here
    pass


# Apply the cleaning pipeline
# TODO: Uncomment and run after implementing the functions above
# df = remove_censored_measurements(df, threshold=IC50_CENSOR_THRESHOLD)
# df = smiles_to_mol(df)
# df = add_basic_descriptors(df)

# TODO: Plot the pKi distribution AFTER cleaning

# TODO: Print basic statistics
# print(df[["pKi", "MW", "logP", "HBD", "HBA"]].describe().round(2))

## 3. Molecular Fingerprints & Chemical Space Visualization

### Morgan Fingerprints (ECFP)

Morgan fingerprints encode circular substructures around each atom:
- **Radius 2** captures atom environments up to 2 bonds away (similar to ECFP4)
- Each substructure is hashed to a bit position in a fixed-length vector
- Similarity between molecules is measured using **Tanimoto coefficient**

### UMAP Projection

UMAP (Uniform Manifold Approximation and Projection) reduces the 2048-dimensional fingerprint space to 2D for visualization:
- Uses Jaccard distance (equivalent to 1 - Tanimoto for binary vectors)
- Preserves local neighborhood structure
- Reveals clusters of similar molecules

---

## 📝 Exercise 2.1: Compute Fingerprints and UMAP Projection

**Task**: Compute Morgan fingerprints for all molecules and visualize chemical space using UMAP.

**Expected Deliverable**:
1. A list of Morgan fingerprints for all molecules
2. A 2D UMAP projection colored by pKi
3. Brief observations about the chemical space structure

In [ ]:
# ============================================================================
# EXERCISE 2.1: Compute fingerprints and UMAP projection
# ============================================================================

def compute_fingerprints(df: pd.DataFrame) -> list:
    """
    Compute Morgan fingerprints for all molecules.
    
    TODO: Implement this function
    - Use AllChem.GetMorganFingerprintAsBitVect(mol, FP_RADIUS, nBits=FP_BITS)
    - Return a list of fingerprints
    """
    # TODO: Your code here
    pass


def umap_projection(fps: list) -> np.ndarray:
    """
    Project fingerprints to 2D using UMAP.
    
    TODO: Implement this function
    - Convert fingerprints to numpy array using fingerprints_to_array()
    - Create a UMAP reducer with metric="jaccard"
    - Return the 2D embedding
    
    Hint: Use umap.UMAP(n_neighbors=15, min_dist=0.1, metric="jaccard", random_state=RANDOM_SEED)
    """
    # TODO: Your code here
    pass


def plot_umap(emb: np.ndarray, values: np.ndarray, title: str) -> None:
    """
    Plot UMAP embedding colored by a value (e.g., pKi).
    
    TODO: Implement this function
    - Use plt.scatter with c=values and a colormap
    - Add colorbar, labels, title
    """
    # TODO: Your code here
    pass


# TODO: Compute fingerprints
# fps = compute_fingerprints(df)

# TODO: Run UMAP projection
# emb = umap_projection(fps)

# TODO: Visualize chemical space
# plot_umap(emb, df["pKi"].values, "Chemical Space (UMAP) — colored by pKi")

## 4. Activity Cliffs

### What are Activity Cliffs?

An **activity cliff** is a pair of molecules that are:
1. **Structurally very similar** (high Tanimoto similarity, e.g., >0.95)
2. **Dramatically different in activity** (large ΔpKi, e.g., >2.0 units = 100× difference in potency)

### Why do Activity Cliffs Matter?

- They reveal **SAR (Structure-Activity Relationship)** hotspots
- Small chemical changes can have huge biological effects
- They are **notoriously difficult for ML models** to predict
- A model that memorizes training data will fail on cliffs in the test set

---

## 📝 Exercise 3.1: Find and Visualize Activity Cliffs

**Task**: Identify pairs of molecules that form activity cliffs and visualize them.

**Expected Deliverable**:
1. A function to find activity cliffs given similarity and ΔpKi thresholds
2. Number of activity cliffs found
3. Visualization of the top 5 most dramatic cliffs (molecule pairs side by side)

In [ ]:
# ============================================================================
# EXERCISE 3.1: Find and visualize activity cliffs
# ============================================================================

def find_activity_cliffs(
    df: pd.DataFrame, 
    fps: list, 
    sim_threshold: float = 0.95, 
    delta_pki_threshold: float = 2.0
) -> list:
    """
    Find pairs of molecules that are structurally similar but have very different activities.
    
    TODO: Implement this function
    - For each pair of molecules, compute Tanimoto similarity
    - If similarity >= sim_threshold AND |ΔpKi| >= delta_pki_threshold, it's a cliff
    - Return list of tuples: (idx_i, idx_j, similarity, delta_pKi)
    
    Hint: Use DataStructs.BulkTanimotoSimilarity(fps[i], fps) to compute similarity
          of molecule i to all others efficiently
    """
    # TODO: Your code here
    pass


def visualize_activity_cliffs(df: pd.DataFrame, cliffs: list, n_pairs: int = 5):
    """
    Display pairs of molecules that form activity cliffs.
    
    TODO: Implement this function
    - Sort cliffs by delta_pKi (most dramatic first)
    - Use MolsToGridImage to display pairs
    - Include pKi values and similarity in legends
    """
    # TODO: Your code here
    pass


# TODO: Find activity cliffs
# cliffs = find_activity_cliffs(df, fps, sim_threshold=0.95, delta_pki_threshold=2.0)

# TODO: Visualize top cliffs
# visualize_activity_cliffs(df, cliffs, n_pairs=5)

## 5. Scaffold Analysis

### Murcko Scaffolds

The **Murcko scaffold** is the core ring system of a molecule (stripping side chains):
- Useful for grouping molecules by chemical series
- Important for **scaffold-based splitting** — ensuring the model sees novel chemotypes at test time

### Why Scaffold Splitting?

Random splits often leak information:
- Training and test sets may contain molecules from the same series
- The model can "cheat" by memorizing series-level patterns
- This leads to **overly optimistic** performance estimates

In [ ]:
# ============================================================================
# SCAFFOLD ANALYSIS (provided)
# ============================================================================

def compute_scaffolds(df: pd.DataFrame) -> pd.DataFrame:
    """Extract Murcko scaffolds for all molecules."""
    df = df.copy()
    df["scaffold"] = df["mol"].apply(
        lambda m: Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(m))
    )
    return df


# TODO: Compute scaffolds (uncomment after Exercise 1.2 is complete)
# df = compute_scaffolds(df)
# n_scaffolds = df["scaffold"].nunique()
# print(f"[INFO] Found {n_scaffolds} unique scaffolds among {len(df)} molecules")
# print(f"       Average molecules per scaffold: {len(df) / n_scaffolds:.1f}")

## 6. Dataset Splitting Strategies

We compare **4 different splitting strategies** to understand how the choice affects model evaluation:

### 1. Random Split
- Molecules are randomly assigned to train/test
- **Problem**: Similar molecules (even from the same series) can appear in both sets
- **Result**: Overly optimistic performance

### 2. Scaffold Split
- Entire scaffolds (chemical series) go to either train OR test
- **Effect**: Tests generalization to new chemotypes
- **Result**: More realistic but still may group similar scaffolds

### 3. Butina Clustering Split
- Clusters molecules by Tanimoto similarity using Butina algorithm
- Entire clusters go to train or test
- **Effect**: Ensures structural novelty in test set
- **Result**: Harder than scaffold split

### 4. UMAP-based Split
- Uses UMAP 2D projection to identify spatially separated regions
- Test set comes from distinct regions of chemical space
- **Effect**: Maximum structural novelty
- **Result**: The hardest, most realistic test of generalization

---

## 📝 Exercise 4.1: Implement Scaffold Split

**Task**: Implement a function that splits data by scaffold.

**Expected Deliverable**: A function that returns train/test boolean masks ensuring no scaffold appears in both sets.

In [ ]:
# ============================================================================
# EXERCISE 4.1: Implement scaffold split
# ============================================================================

def random_split(X: np.ndarray, y: np.ndarray, test_size: float = 0.2):
    """Standard random train/test split (provided)."""
    return train_test_split(X, y, test_size=test_size, random_state=RANDOM_SEED)


def scaffold_split(df: pd.DataFrame, test_size: float = 0.2):
    """
    Split by scaffold: entire chemical series go to train or test.
    
    TODO: Implement this function
    - Get unique scaffolds from df["scaffold"]
    - Shuffle them randomly
    - Assign first test_size fraction of scaffolds to test set
    - Return (train_mask, test_mask) as boolean numpy arrays
    
    Returns:
        train_mask: boolean array, True for training samples
        test_mask: boolean array, True for test samples
    """
    # TODO: Your code here
    pass

---

## 📝 Exercise 4.2: Implement Butina Clustering Split

**Task**: Implement a function that clusters molecules using Butina algorithm and splits by clusters.

**Expected Deliverable**: A function that returns train/test boolean masks ensuring no cluster appears in both sets.

**Hints**:
- Use `DataStructs.BulkTanimotoSimilarity` to compute pairwise similarities
- Use `Butina.ClusterData(distances, n_molecules, cutoff, isDistData=True)`
- The Butina algorithm needs a distance matrix in condensed form (lower triangle)

In [ ]:
# ============================================================================
# EXERCISE 4.2: Implement Butina clustering split
# ============================================================================

def butina_split(fps: list, cutoff: float = 0.6, test_size: float = 0.2):
    """
    Cluster molecules using Butina algorithm, then split by clusters.
    
    TODO: Implement this function
    - Compute pairwise Tanimoto distances (1 - similarity)
    - Cluster using Butina.ClusterData()
    - Shuffle clusters and assign test_size fraction to test set
    - Return (train_mask, test_mask) as boolean numpy arrays
    
    Args:
        fps: list of fingerprints
        cutoff: distance cutoff for clustering (default 0.6 = 40% similarity)
        test_size: fraction of clusters to use for test
        
    Returns:
        train_mask: boolean array, True for training samples
        test_mask: boolean array, True for test samples
    """
    # TODO: Your code here
    pass

In [ ]:
# ============================================================================
# UMAP-BASED SPLIT (provided)
# ============================================================================

def umap_split(emb: np.ndarray, test_size: float = 0.2, n_regions: int = 10):
    """
    Split by UMAP regions: divide 2D space into grid cells, 
    assign entire cells to train or test.
    
    This ensures test molecules come from distinct regions of chemical space.
    """
    n = len(emb)
    
    # Create grid-based regions
    x_bins = np.linspace(emb[:, 0].min(), emb[:, 0].max(), n_regions + 1)
    y_bins = np.linspace(emb[:, 1].min(), emb[:, 1].max(), n_regions + 1)
    
    # Assign each point to a region
    x_idx = np.digitize(emb[:, 0], x_bins[:-1]) - 1
    y_idx = np.digitize(emb[:, 1], y_bins[:-1]) - 1
    region_ids = x_idx * n_regions + y_idx
    
    # Get unique regions and shuffle
    unique_regions = np.unique(region_ids)
    np.random.shuffle(unique_regions)
    
    # Assign regions to test set until we reach target size
    test_mask = np.zeros(n, dtype=bool)
    current_test_size = 0
    target_test_size = int(test_size * n)
    
    for region in unique_regions:
        if current_test_size >= target_test_size:
            break
        region_mask = region_ids == region
        test_mask |= region_mask
        current_test_size = test_mask.sum()
    
    return ~test_mask, test_mask


print("✅ UMAP split function provided")

## 7. QSAR Modeling

### Model: XGBoost Regressor

We use **XGBoost** (Gradient Boosted Trees) for QSAR modeling:
- Works well with sparse fingerprint features
- Handles non-linear relationships
- Fast training on medium-sized datasets

### Evaluation Metrics

1. **RMSE** (Root Mean Squared Error): Measures prediction accuracy in pKi units
2. **Spearman ρ** (Rank Correlation): Measures ability to rank compounds correctly
   - More important for virtual screening where we care about ranking, not absolute values

---

## 📝 Exercise 4.3: Compare All 4 Splitting Strategies

**Task**: Train XGBoost models using all 4 splits and compare their performance.

**Expected Deliverable**:
1. A comparison table showing RMSE and Spearman ρ for each split
2. Bar chart visualization of the results
3. Identification of which split is hardest/easiest

In [ ]:
# ============================================================================
# QSAR MODELING FUNCTIONS (provided)
# ============================================================================

def train_xgb(X_train: np.ndarray, y_train: np.ndarray) -> XGBRegressor:
    """Train an XGBoost regressor."""
    model = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        random_state=RANDOM_SEED,
        verbosity=0
    )
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: XGBRegressor, X_test: np.ndarray, y_test: np.ndarray) -> dict:
    """Evaluate model and return metrics."""
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rho, pval = spearmanr(y_test, y_pred)
    
    return {
        "rmse": rmse,
        "spearman_rho": rho,
        "y_pred": y_pred,
        "y_test": y_test
    }


print("✅ Modeling functions provided")

In [ ]:
# ============================================================================
# EXERCISE 4.3: Compare all 4 splitting strategies
# ============================================================================

# TODO: Prepare feature matrix and target
# X = fingerprints_to_array(fps)
# y = df["pKi"].values

# TODO: Store results for comparison
# results = {}

# -------------------------------------------------------------------------
# TODO: 1. RANDOM SPLIT
# -------------------------------------------------------------------------
# X_train, X_test, y_train, y_test = random_split(X, y)
# model_random = train_xgb(X_train, y_train)
# results["Random"] = evaluate_model(model_random, X_test, y_test)
# print(f"Random: RMSE={results['Random']['rmse']:.3f}, ρ={results['Random']['spearman_rho']:.3f}")

# -------------------------------------------------------------------------
# TODO: 2. SCAFFOLD SPLIT
# -------------------------------------------------------------------------
# train_mask, test_mask = scaffold_split(df)
# X_train, X_test = X[train_mask], X[test_mask]
# y_train, y_test = y[train_mask], y[test_mask]
# model_scaffold = train_xgb(X_train, y_train)
# results["Scaffold"] = evaluate_model(model_scaffold, X_test, y_test)
# print(f"Scaffold: RMSE={results['Scaffold']['rmse']:.3f}, ρ={results['Scaffold']['spearman_rho']:.3f}")

# -------------------------------------------------------------------------
# TODO: 3. BUTINA CLUSTERING SPLIT
# -------------------------------------------------------------------------
# train_mask, test_mask = butina_split(fps, cutoff=0.6)
# X_train, X_test = X[train_mask], X[test_mask]
# y_train, y_test = y[train_mask], y[test_mask]
# model_butina = train_xgb(X_train, y_train)
# results["Butina"] = evaluate_model(model_butina, X_test, y_test)
# print(f"Butina: RMSE={results['Butina']['rmse']:.3f}, ρ={results['Butina']['spearman_rho']:.3f}")

# -------------------------------------------------------------------------
# TODO: 4. UMAP-BASED SPLIT
# -------------------------------------------------------------------------
# train_mask, test_mask = umap_split(emb, n_regions=8)
# X_train, X_test = X[train_mask], X[test_mask]
# y_train, y_test = y[train_mask], y[test_mask]
# model_umap = train_xgb(X_train, y_train)
# results["UMAP"] = evaluate_model(model_umap, X_test, y_test)
# print(f"UMAP: RMSE={results['UMAP']['rmse']:.3f}, ρ={results['UMAP']['spearman_rho']:.3f}")

---

## 📝 Exercise 5.1: Analyze Results and Conclusions

**Task**: Create visualizations comparing the 4 splits and identify key insights.

**Expected Deliverable**:
1. A comparison DataFrame sorted by difficulty (highest RMSE first)
2. Bar charts showing RMSE and Spearman ρ for each split
3. Scatter plots of predicted vs. experimental values for each split
4. Written answers to the following questions:

### Questions to Answer:

1. **Which split was hardest? Which was easiest?** Why do you think this is?

2. **What is the performance gap** between random and the hardest split? What does this tell you about using random splits for QSAR evaluation?

3. **How do activity cliffs relate** to the difficulty of structure-aware splits?

4. **What recommendations** would you give for evaluating QSAR models in a real drug discovery project?

In [ ]:
# ============================================================================
# EXERCISE 5.1: Analyze results and create visualizations
# ============================================================================

# TODO: Create a comparison DataFrame
# comparison_df = pd.DataFrame({
#     "Split": list(results.keys()),
#     "RMSE": [results[k]["rmse"] for k in results],
#     "Spearman ρ": [results[k]["spearman_rho"] for k in results]
# }).sort_values("RMSE", ascending=False)
# print(comparison_df)

# TODO: Identify hardest and easiest splits
# hardest = comparison_df.iloc[0]["Split"]
# easiest = comparison_df.iloc[-1]["Split"]
# print(f"\nHardest: {hardest}, Easiest: {easiest}")

# TODO: Create bar charts comparing RMSE and Spearman ρ


# TODO: Create scatter plots for each split (2x2 grid)


# TODO: Calculate and print the performance gap
# gap_rmse = comparison_df.iloc[0]["RMSE"] - comparison_df.iloc[-1]["RMSE"]
# print(f"RMSE gap: {gap_rmse:.3f}")

### Your Answers:

**1. Which split was hardest? Which was easiest? Why?**

*TODO: Write your answer here*



**2. What is the performance gap? What does this tell you?**

*TODO: Write your answer here*



**3. How do activity cliffs relate to split difficulty?**

*TODO: Write your answer here*



**4. Recommendations for real QSAR evaluation?**

*TODO: Write your answer here*

---

## 8. Summary

### Key Takeaways

1. **Splitting strategy dramatically affects apparent model performance**
   - Random splits are overly optimistic
   - Structure-aware splits (scaffold, Butina, UMAP) give more realistic estimates

2. **Activity cliffs are a fundamental challenge**
   - Small structural changes can cause large activity differences
   - Fingerprint-based models struggle with cliffs
   - This is a key limitation of 2D representations

3. **The "applicability domain" matters**
   - Models work best within the chemical space of training data
   - Predictions for novel chemotypes should be treated with caution

### Recommendations for Real-World QSAR

- Always use **structure-aware splitting** for validation
- Report performance on multiple split types
- Be skeptical of models with very high random-split performance
- Consider the **applicability domain** when deploying models
- Use **ensemble methods** to estimate prediction uncertainty

---

## ✅ Checklist

Before submitting, make sure you have completed:

- [ ] Exercise 1.1: Identified censoring threshold from data
- [ ] Exercise 1.2: Implemented data cleaning functions
- [ ] Exercise 2.1: Computed fingerprints and UMAP projection
- [ ] Exercise 3.1: Found and visualized activity cliffs
- [ ] Exercise 4.1: Implemented scaffold split
- [ ] Exercise 4.2: Implemented Butina clustering split
- [ ] Exercise 4.3: Compared all 4 splitting strategies
- [ ] Exercise 5.1: Analyzed results and answered questions

---

## 📝 Exercise 1.2: Clean Data and Compute Descriptors

**Task**: Remove censored measurements, parse SMILES to RDKit molecules, and add molecular descriptors.

**Expected Deliverable**:
1. A cleaned DataFrame with only non-censored molecules
2. RDKit molecule objects in a "mol" column
3. Basic molecular descriptors (MW, logP, HBD, HBA)
4. A histogram showing the pKi distribution AFTER removing censored values

**Functions to implement**:
- `remove_censored_measurements(df, threshold)`: Remove rows with IC50 > threshold
- `smiles_to_mol(df)`: Parse SMILES strings to RDKit molecules
- `add_basic_descriptors(df)`: Add MW, logP, HBD, HBA columns